In [1]:
# ============================
# 1. 필수 라이브러리 설치
# ============================
!pip -q install langchain langchain-community langchain-text-splitters faiss-cpu chromadb sentence-transformers pypdf rank_bm25 lark

In [5]:
# ============================
# 2. GitHub에서 PDF 다운로드
# ============================
import requests, os
import time

url = "https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%90%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf"
pdf_path = "2020_경제금융용어 700선_게시.pdf"

# Add retry mechanism for downloading the PDF
max_retries = 3
for attempt in range(max_retries):
    if not os.path.exists(pdf_path) or os.path.getsize(pdf_path) == 0: # Check if file exists or is empty
        try:
            print(f"Downloading PDF (Attempt {attempt + 1}/{max_retries}):", url)
            r = requests.get(url, stream=True)
            r.raise_for_status() # Raise an exception for bad status codes
            with open(pdf_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
            if os.path.exists(pdf_path) and os.path.getsize(pdf_path) > 0:
                print(" PDF downloaded successfully:", pdf_path)
                break # Exit loop if download is successful
        except requests.exceptions.RequestException as e:
            print(f" Download failed: {e}")
            time.sleep(2) # Wait before retrying
    else:
        print(" PDF already exists and is not empty:", pdf_path)
        break # Exit loop if file already exists and is not empty
else:
    print(f" Failed to download PDF after {max_retries} attempts.")
    # You might want to exit or raise an error here if the download is critical
    # For now, we'll continue with the potentially incomplete/corrupted file

# ============================
# 3. PDF 로드 & 청크 분할
# ============================
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(" 전체 문서 수:", len(docs))
print(" 청크 분할 후:", len(chunks))
print("샘플 청크:\n", chunks[0].page_content[:300])

# ============================
# 4. Embedding + VectorStore
# ============================
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_store = FAISS.from_documents(chunks, embedding=embedding)

vector_retriever = faiss_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

query = "인플레이션이 무엇인가요?"
docs = vector_retriever.get_relevant_documents(query)

print("\n🔹 VectorStore 결과:")
for d in docs:
    print("-", d.page_content[:120].replace("\n"," "), d.metadata)

# ============================
# 5. Ensemble Retriever (BM25+FAISS)
# ============================
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever # Corrected import

bm25 = BM25Retriever.from_documents(chunks)
bm25.k = 3

ensemble_retriever = EnsembleRetriever(retrievers=[bm25, vector_retriever], weights=[0.4, 0.6])

docs = ensemble_retriever.get_relevant_documents("인플레이션이 무엇인가요?")
print("\n🔹 Ensemble Retriever 결과:")
for d in docs:
    print("-", d.page_content[:120].replace("\n"," ",), d.metadata)


 PDF already exists and is not empty: 2020_경제금융용어 700선_게시.pdf
 전체 문서 수: 371
 청크 분할 후: 1117
샘플 청크:
 iii
찾아보기
한국은행은 국민들이 경제 및 금융에 대한 이해도를 높이고 경제에 관한 합리적인 
의사결정 능력을 키울 수 있도록 현장 경제교육, 온라인 경제교육, 경제교육 콘텐츠 
개발 등 대국민 경제교육을 다양하게 수행해 오고 있습니다 . 
이의 일환으로 2018년 
 경제금융용어 700선
 을 발간하였는데 그간 동 책자에 대한 
수요가 꾸준히 늘어남에 따라 이번에 추가로 발간하게 되었습니다 .
지난번 내용과 같이 통화정책, 실물경제, 금융안정, 지급결제 등 한국은행 주요 
업무를 이해하는데 필요한 전문 용어와 경제·금융 흐름을 

🔹 VectorStore 결과:
- 예대금리차는 간단히 표현하면 대출금리(수입이자/대출금)에서 예금금리(지급이자/예 수금)를 차감한 것으로, 분석목적에 따라 신규취급액 또는 잔액을 기준으로 측정할 {'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'page': 210, 'page_label': '211'}
- 를 하는 수입국은 해당 물품의 수출국에 적절한 보상을 해줄